Import required dependencies

In [20]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data

In [21]:
# Import functions from Pieter's utils package.
# from utils_pieter import f_describe, f_info, f_get_account_name

load config files

In [22]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)



API configurations

In [23]:
# Load config values
config = load_config(CONFIG_FILE)
print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["settings"]["download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}


api_params={
    "point": "0",
    "type": "2",
    "granularity": "3",
    "granularitytimezone": "1",
    "classification": "2",
    "activity": "1",
    "validfrom[strictly_before]": "2025-02-22",
    "validfrom[after]": "2025-02-01"
    }


{'KNMI': {'api_endpoint': 'https://api.dataplatform.knmi.nl/open-data/v1/datasets/knmi14_globale_straling/versions/3.2/files', 'api_key': 'nctBcxmIU5Ox5CxqzBuiUxLDoDC4yNabewk63UGW', 'notification_key': 'eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjM3ODk5YjE4NjM5ZDQ0ZmRhYjMzYjdhMjBlZDE3Mjg3IiwiaCI6Im11cm11cjEyOCJ9'}, 'ned': {'ned_api_endpoint': 'https://api.ned.nl/v1/utilizations', 'demo-ned-api-key': '01dd5b246eb73805193aa271dfe3ada2589d20f60838d091077155ad4658ac84', 'ned_download_dir': '../../src/data/weather/'}, 'settings': {'download_dir': 'downloads'}}


make API call

In [ ]:
"""Download a data file from an API and save it in the specified directory."""
response = requests.get(API_URL, headers=api_headers, params=api_params)

# Convert JSON to Pandas DataFrame
    
if response.status_code == 200:
    # JSON in bytes format
    json_bytes = response.content

    # Convert bytes → string → dictionary
    json_dict = json.loads(json_bytes.decode("utf-8"))

    # Convert to DataFrame
    df = pd.DataFrame.from_dict(json_dict, orient="index")

    # Convert the list of utilizations into a DataFrame
    json_data = pd.DataFrame(json_dict["hydra:member"])

    # Display the DataFrame as a table
    # print("\nConverted JSON Payload to Table Format:\n")
    # print(utilizations.to_string(index=False))

    # f_describe(df_input = utilizations, n_top = 3)
    json_data.describe()

           
    #file_path = os.path.join(DOWNLOAD_DIR, "downloaded_data.json")  # Change extension as needed
    #with open(file_path, "wb") as file:
    #    file.write(response.content)
    #print(f"File downloaded successfully: {file_path}")
else:
    print(f"Error: {response.status_code} - {response.text}")


File downloaded successfully: downloads/downloaded_data.json


Descriptive analytics using Pieter package

In [25]:
# f_describe(df, 10)
df_orig_num    = json_data.select_dtypes(include='number')
l_df_num_names = df_orig_num.columns.tolist()

print(l_df_num_names)
print(f"\nNumber of numerical variables: {len(l_df_num_names)}")

['id', 'capacity', 'volume', 'percentage', 'emission', 'emissionfactor']

Number of numerical variables: 6


Save as pickel file for data

In [26]:
# Create dictionary 'dc_ned_json_data_1' with objects that will be used in the next exercises.
dc_ned_json_data_1 = {
   'df_orig': json_data    
}

# Save dc_exercise_1_2_3 as 'dc_ned_json_data_1.pkl'
with open('../data/dc-ned-json-data-1.pkl', 'wb') as pickle_file:
    pickle.dump(dc_ned_json_data_1, pickle_file)